This notebook generates the paragraph about the microCT-scanning from logfiles of the scans.

In [29]:
import platform
import os
import pandas
import glob

In [30]:
from parsing_functions import *

In [31]:
# Different locations if running either on Linux or Windows
if 'Linux' in platform.system():
    BasePath = os.path.join(os.path.sep, 'home', 'habi', 'P')
elif 'Windows' in platform.system():
    BasePath = os.path.join('P:', os.sep)
# Use *this* folder for the bone microvasculature manuscript
Root = os.path.join(BasePath, 'Documents', 'Publications', 'Ruslan Bone', 'manubot', 'content', 'data')
print('We are loading all the data from the folder %s' % Root)

We are loading all the data from the folder /home/habi/P/Documents/Publications/Ruslan Bone/manubot/content/data


In [32]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [33]:
# Get *all* log files
# Using os.walk is way faster than using recursive glob.glob, see DataWrangling.ipynb for details
# Not sorting the found logfiles is also making it quicker
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Root)
                   for name in files
                   if name.endswith((".log"))]

In [36]:
print('We found %s log files in %s' % (len(Data), Root))

We found 54 log files in /home/habi/P/Documents/Publications/Ruslan Bone/manubot/content/data


Parse the data from all the log files

In [45]:
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['ControlSoftware'] = [controlsoftware(log) for log in Data['LogFile']]

In [46]:
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Voxelsize_rounded'] = [pixelsize(log,rounded=True) for log in Data['LogFile']]

In [47]:
Data['Source'] = [source(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]
Data['Exposure'] = [exposuretime(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]

In [48]:
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]

In [49]:
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['NumProj'] = [numproj(log) for log in Data['LogFile']]
Data['ProjSize'] = [projection_size(log) for log in Data['LogFile']]
Data['ThreeSixty'] = [threesixtyscan(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Wide'] = [overlapscan(log) for log in Data.LogFile]
Data['Duration'] = [duration(log) for log in Data['LogFile']]
Data['Date'] = [scandate(log) for log in Data['LogFile']]

In [51]:
Data['NRecon'] = [nreconversion(log)[1] for log in Data['LogFile']]
Data['RingRemoval'] = [ringremoval(log) for log in Data['LogFile']]
Data['Beamhardening'] = [beamhardening(log) for log in Data['LogFile']]
Data['DefectPixelMasking'] = [defectpixelmasking(log) for log in Data['LogFile']]
Data['GrayValue'] = [reconstruction_grayvalue(log) for log in Data['LogFile']]

In [52]:
Data['RecSize'] = [reconstruction_size(log) for log in Data['LogFile']]
Data['ROI'] = [region_of_interest(log, verbose=False) for log in Data['LogFile']]

In [53]:
Data['Duration'] = [duration(log) for log in Data['LogFile']]
Data['Date'] = [scandate(log) for log in Data['LogFile']]

----

Now that we loaded the data, we customize the standard log file parser notebook for the microvasculature manuscript.

In [57]:
# Extract folder name
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [58]:
# Generate sample name to then match to figure number
# We bluntly split the path at the `os.path.sep` and user the first item of this separated list
Data['Sample'] = [(foldername[len(Root)+1:]).split(os.path.sep)[0] for foldername in Data['Folder']]

In [59]:
def match_to_figure(samplename):
    figurenumber = None
    if 'Israel' in samplename:
        figurenumber = 1
    elif '11O' in samplename:
        # Some are named 11OKT, some 11Okt
        figurenumber = 2
    elif 'c1m5' in samplename:
        figurenumber = 3
    elif '0_99' in samplename:
        figurenumber = 4    
    elif 'Vreni' in samplename:
        figurenumber = 5
    elif '_sample4' in samplename:
        figurenumber = 6
    elif 'Mouse_1EAR' in samplename:
        figurenumber = 7
    return(str(figurenumber))

In [60]:
# Map sample names to our figures 
Data['Figure'] = [match_to_figure(s) for s in Data['Sample']]

In [61]:
# Sort dataframe by figure number
Data.sort_values(by=['Figure', 'Sample'], inplace=True)
# Reset dataframe index
Data = Data.reset_index(drop=True)

In [62]:
# Cull irrelevant beginning of path from logfile
Data['LogFile'] = [lf[len(Root)+1:] for lf in Data['LogFile']]

In [64]:
# Cull irrelevant beginning of path from logfile
Data['Folder'] = [folder.split('data')[1] for folder in Data['Folder']]

In [66]:
Data.sample()

,LogFile,Scanner,Software,Voxelsize,Voxelsize_rounded,Source,Camera,Exposure,Averaging,Voltage,...,RecSize,ROI,RecRotation,ControlSoftware,Duration,Date,NRecon,Folder,Sample,Figure
39,Vreni_Jaw_Jul19_AL1mm-Cu02mm_100kV_9um_OFFSET_...,SkyScan 1273,1.1,9.000382,9.0,Hamamatsu L9181-02,"DEXELA-2315[v1], S/N 32960",225,5,100.0,...,"(4724, 3012)","(4152, 1138, 10, 4734)",-85.59,1.1,14868.0,2019-07-20 14:23:14,1.7.4.2,/Vreni_Jaw_Jul19_AL1mm-Cu02mm_100kV_9um_OFFSET...,Vreni_Jaw_Jul19_AL1mm-Cu02mm_100kV_9um_OFFSET_,5


In [69]:
# Save out relevant columns of the dataframe as csv for adding to the supplementary materials
# With renamed column names
Data[['Figure', 'Sample', 'Scanner', 'ControlSoftware', 'Date',
      'Voxelsize_rounded', 'Source', 'Voltage', 'Current', 'Filter', 'ProjSize', 'ThreeSixty', 'RotationStep',
      'Averaging', 'Exposure', 'Stacks', 'Wide', 'Duration',
      'NRecon', 'RingRemoval', 'Beamhardening', 'LogFile'
     ]].to_csv(os.path.join(Root, 'ScanningDetails.csv'),
               index=False,
               header=['Figure', 'Sample name', 'Scanner', 'Control software version', 'Scan date',
                       'Voxelsize [μm]', 'X-ray source', 'Source voltage [kV]', 'Source current [μA]',
                       'Filter', 'Projection size', '360°-scan', 'Rotation step [°]', 'Frame averaging',
                       'Exposure time [ms]', 'Stacked scans', 'Overlap scans',
                       'Scan duration [s]',
                       'NRecon version', 'Ring removal correction', 'Beam hardening correction',
                       'Log file'
                     ])
# This csv file is nicely shown online on GitHub

In [71]:
# Save out as .xlsx sheet, too
# This Excel sheet is not uploaded to Github, but makes it easy to quickly look at it
Data[['Figure', 'Sample', 'Scanner', 'ControlSoftware', 'Date',
      'Voxelsize_rounded', 'Source', 'Voltage', 'Current', 'Filter', 'ProjSize', 'ThreeSixty', 'RotationStep',
      'Averaging', 'Exposure', 'Stacks', 'Wide', 'Duration',
      'NRecon', 'RingRemoval', 'Beamhardening', 'LogFile'
     ]].to_excel(os.path.join(Root, 'ScanningDetails.xlsx'),
               index=False,
               header=['Figure', 'Sample name', 'Scanner', 'Control software version', 'Scan date',
                       'Voxelsize [μm]', 'X-ray source', 'Source voltage [kV]', 'Source current [μA]',
                       'Filter', 'Projection size', '360°-scan', 'Rotation step [°]', 'Frame averaging',
                       'Exposure time [ms]', 'Stacked scans', 'Overlap scans',
                       'Scan duration [s]',
                       'NRecon version', 'Ring removal correction', 'Beam hardening correction',
                       'Log file'
                     ])

----

In [73]:
# Generate text in the manuscript

In [75]:
for figure in Data.Figure.unique():
    print(figure)

1
2
3
4
5
6
7


In [ ]:
Data.Current.mean()

In [ ]:
Data.Filter.unique()

In [ ]:
Data.Filter.unique().any()

In [ ]:
print('The X-ray source was set to a tube voltage of', 
      " OR ".join(str(value) for value in Data.Voltage.unique()),
      'kV and a tube current of',
      " OR ".join(str(value) for value in Data.Current.unique()),
      'µA, the x-ray spectrum was', end=' ')
if Data.Filter.unique().any():
    print('filtered by', " OR ".join(str(value) for value in Data.Filter.unique()), end=' ')
else:
    print('not filtered', end=' ')
print('prior to incidence onto the sample.')

In [ ]:
# TODO: Flip the text of the filter to make it nicer

In [ ]:
Data.Wide.unique()

In [ ]:
Data.ProjSize.unique()

In [ ]:
Data.ThreeSixty.unique()

In [ ]:
print('For each sample, we recorded a set of', end=' ')
if Data.Filter.unique().tolist():   
    print(" or ".join(str(value) for value in Data.Stacks.unique()),
          'stacked scans overlapping the sample height, each stack was recorded with', end=' ')
print(" or ".join(str(value) for value in Data.NumProj.unique()), 'projections of', end=' ')
for cs in Data.ProjSize.unique():
    print(cs[0], end=' ')
print('x', end=' ')
for cs in Data.ProjSize.unique():
    print(cs[1], end=' ')
print('pixels', end=' ')
if Data.Wide.unique().tolist():
    print('(' + " or ".join(str(value) for value in Data.Wide.unique()), 'projections stitched laterally)', end=' ')
print('at every',
       str(" or ".join(str(value) for value in Data.RotationStep.unique())) + '° over a ', end='')
if Data.ThreeSixty.unique().tolist():
     print('360°', end=' ')
else:
    print('180°', end=' ')
print('sample rotation.')

In [ ]:
Data.Exposure.mean()

In [ ]:
print('Every single projection was exposed for',
      " or ".join(str(value) for value in Data.Exposure.unique()),
      'ms,',
      " or ".join(str(value) for value in Data.Averaging.unique()),
      'projections were averaged to one to greatly reduce image noise.')

In [ ]:
log=Data.LogFile.sample(n=1)

In [ ]:
# For the cell below, we exclude all subscan logfiles
# These do *not* contain any information on NRecon
# From this subset, we select 1 logfile and work with that
log = Data[~ Data['LogFile'].str.contains('~')].sample(n=1).LogFile

In [ ]:
log = log[log.index[0]]

In [ ]:
os.path.basename(log)

In [ ]:
print('This resulted in a scan time of approximately ', end='')
if duration(log)/3600 > 1:
    # Scan took hours
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
else:
    print(timeformat(datetime.timedelta(seconds=duration(log)),
                     '{minutes} minutes'), end=' ')
if not stacks(log) == 1:
    print('per stack and about',
          timeformat(stacks(log) * datetime.timedelta(seconds=duration(log)),
                     '{hours} hours and {minutes} minutes'), end=' ')
print('per sample', end='')
if stacks(log) == 1:
    print('.')
else:
    print(' (with', stacks(log), 'stacks).')

In [ ]:
print('In total, we scanned', Data.Stacks.sum(), 'stacks.')
print('Each stack took approximately',
      Data.Duration.mean() // 60,
      'minutes (' + str(datetime.timedelta(seconds=Data.Duration.mean())) + ')')
print('In total, we thus scanned for about', 
      timeformat(Data.Stacks.sum() *
                 datetime.timedelta(seconds=Data.Duration.mean()),
                 '{days} days, {hours} hours and {minutes} minutes.'))
hourlyrate = 125
print('At the MIC rate of %s CHF/h, this would have cost %s CHF' % (
    hourlyrate,
    int(round(Data.Stacks.sum() * Data.Duration.mean() / 60 / 60 * hourlyrate))))

In [ ]:
Data.Voxelsize.mean()

In [ ]:
Data.Beamhardening.unique()

In [ ]:
print('The projection images were then subsequently reconstructed into a 3D stack',
      'of images with',
      Data.Version.unique()[0][0],
      '(Version',
      nreconversion(log)[1] + ', Bruker microCT, Kontich Belgium)', end=' ')
if ringremoval(log):
    print('using a ring artifact correction of',
          ringremoval(log), end='')
if beamhardening(log):
    print(' and a beam hardening correction of',
          beamhardening(log),
          '%.')
else:
    print('.')
print('The whole process resulted in datasets with an isometric voxel size of',
      " or ".join(str(value) for value in Data.Voxelsize_rounded.unique()),
      'µm.')    